#                                                       The Black-Scholes PDE

### Imports:

In [5]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
%matplotlib inline

from scipy import sparse
from scipy.sparse.linalg import splu
from scipy.sparse.linalg import spsolve

from IPython.display import display
import sympy 
sympy.init_printing()

### Model Numerical Solution Class:

In [ ]:
class BSM:
    def __init__(self, r, sigma,K, S_0,period, space_steps=3000, time_steps=2000, initial_time=0) -> None:
        self.__r = r
        self.__sigma = sigma
        self.__K = K
        self.__period = period
        self.__S_0 = S_0
        self.__x_0 = np.log(self.__S_0)
        self.__time_steps = time_steps
        self.__space_steps = space_steps
        self.__S_max = 3*self.__K 
        self.__S_min = self.__K/3 
        self.__x_max = np.log(self.__S_max)
        self.__x_min = np.log(self.__S_min)
        self.__X, self.__dx = np.linspace(self.__x_min, self.__x_max, self.__space_steps, retstep=True)
        self.__T, self.__dt = np.linspace(initial_time, self.__period, self.__time_steps, retstep=True)

    def calc_coeff(self):
        sig2 = self.__sigma*self.__sigma; dxx = self.__dx * self.__dx

        a = ( (self.__dt/2) * ( (self.__r-0.5*sig2)/self.__dx - sig2/dxx ) )
        b = ( 1 + self.__dt * ( sig2/dxx + self.__r ) )
        c = (-(self.__dt/2) * ( (self.__r-0.5*sig2)/self.__dx + sig2/dxx ) )

        return a, b, c

    def coeff_matrix(self):
        a, b, c = self.calc_coeff()

        D = sparse.diags([a, b, c], [-1, 0, 1], shape=(self.__space_steps-2, self.__space_steps-2)).tocsc()
        return D

    def solve_call(self):
        V = np.zeros((self.__space_steps, self.__time_steps))
        #terminal condition
        Payoff = np.maximum(np.exp(self.__X)-self.__K, 0)
        V[:, -1] = Payoff
        #boundary conditions
        V[0, :] = 0
        V[-1, :] = np.exp(self.__x_max) - self.__K * np.exp(-self.__r* self.__T[::-1] )
        offset = np.zeros(self.__space_steps-2)
        a, b, c = self.calc_coeff()
        D = self.coeff_matrix()
        
        for i in range(self.__time_steps-2,-1,-1):
            offset[0] = a * V[0,i]
            offset[-1] = c * V[-1,i]; 
            V[1:-1,i] = spsolve( D, (V[1:-1,i+1] - offset) ) 

    def solve_put(self):
        V = np.zeros((self.__space_steps, self.__time_steps))
        #terminal condition
        Payoff = np.maximum(self.__K-np.exp(self.__X), 0)
        V[:, -1] = Payoff
        #boundary conditions
        V[-1, :] = 0
        V[0, :] = self.__K * np.exp(-self.__r* self.__T[::-1] )
        offset = np.zeros(self.__space_steps-2)
        a, b, c = self.calc_coeff()
        D = self.coeff_matrix()
        
        for i in range(self.__time_steps-2,-1,-1):
            offset[0] = a * V[0,i]
            offset[-1] = c * V[-1,i]; 
            V[1:-1,i] = spsolve( D, (V[1:-1,i+1] - offset) ) 







In [18]:
x = np.zeros((3,4))
x[0,0] = 1
x[1,1] = 2
x[2,2] = 3
x[1,:]

array([0., 2., 0., 0.])

In [19]:
np.zeros(4)

array([0., 0., 0., 0.])

In [20]:
np.zeros((5,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])